In [32]:
import os, sys, json, datetime, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
from tqdm import tqdm

In [33]:
from cprl_functions.defined_functions import create_pk, extract_title_and_name, get_recent_file

In [34]:
from bs4 import BeautifulSoup
from lxml import etree, html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, extract_title_and_name, get_recent_file
from unidecode import unidecode

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [35]:
## Driver Set up
webdriver_path = r"C:\Users\clutz\hunt_env\chrome driver\chromedriver-win64\chromedriver.exe"
chrome_options = Options()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
# Set up WebDriver service
service = Service(webdriver_path)

#call on driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)


In [39]:

nd_urls = """https://ndlegis.gov/assembly/69-2025/committees/house/appropriations-education-and-environment-division
https://ndlegis.gov/assembly/69-2025/committees/house/education
https://ndlegis.gov/assembly/69-2025/committees/house/human-services
https://ndlegis.gov/assembly/69-2025/committees/senate/appropriations-education-and-environment-division
https://ndlegis.gov/assembly/69-2025/committees/senate/appropriations-human-resources-division
https://ndlegis.gov/assembly/69-2025/committees/senate/education"""
nd_urls = [x for x in nd_urls.split('\n') if len(x) != 0]


In [40]:
#house
oh_coms = []
for url in nd_urls:
    # print(f'going to {url}')
    driver.get(url)
    # divs = driver.find_elements(By.CLASS_NAME, "image-wrapper member-image-wrapper")
    
    html_from_page = driver.page_source
    soup = BeautifulSoup(html_from_page, 'html.parser')
    # divs = soup.find_all("div", {'class':'image-wrapper member-image-wrapper'})
    divs = soup.find_all('a', href=True)

    print(len(divs))
    members_bucket = []
    for x in divs:
        if '/biography/' in str(x.get("href")).lower() and not 'Photo' in str(x):#  and 'target="_self"' in str(x).lower():
            print((x.get_text()))
            
        


184
Mike Nathe
Steve Swiontek
Karla Rose Hanson
Scott Louser
Bob Martinson
David Richter
Mark Sanford
213
Pat D. Heinert
Cynthia Schreiber-Beck
Liz Conmy
LaurieBeth Hager
Patrick R. Hatlestad
Dori Hauck
Matthew Heilman
Jim Jonas
Donald W. Longmuir
Roger A. Maki
Andrew Marschall
Desiree Morton
Anna S. Novak
Doug Osowski
209
Matthew Ruby
Kathy Frelich
Karen A. Anderson
Mike Beltz
Macy Bolinske
Jayme Davis
Gretchen Dobervich
Clayton Fegley
Jared Hendrix
Dawson Holle
Dwight Kiefert
Nico Rios
Karen M. Rohr
176
Ronald Sorvaag
Cole Conley
Scott Meyer
Donald Schaible
Paul J. Thomas
176
Dick Dever
Sean Cleary
Kyle Davison
Jeffery J. Magrum
Tim Mathern
181
Todd Beard
Randy D. Lemm
Michelle Axtman
Josh Boschee
Justin Gerhardt
Mike Wobbema


In [ ]:
        if '/biography/' in str(x.get("href")).lower():#  and 'target="_self"' in str(x).lower():
            print(x.text)
            # text = x.text.encode('unicode_escape').decode('utf-8')
            text_split = x.text.split('\n')

            # print(text_split)

            found_position = False
            for i,b in enumerate(text_split):
                position = np.nan
                words = ['chair', 'vice', 'ranking', 'member']
                
                #covers len of 0
                if len(b) == 0:
                    continue
                            #covers len of 0
                elif '\xa0' in str(b).strip() and len(str(b).strip())<5:
                    continue

                #covers district sections
                elif 'district' in str(b).lower().strip():
                    match = re.search(r"[Dd]istrict\s+(\d+)", str(b))

                    if match:
                        district_number = match.group(1)  # Extract the number
                        # print(district_number)  # Output: 36
                    else:
                        print("No match found")
                #covers positions
                elif any(word in str(b).lower().strip() for word in words):
                    position_ext = str(b)
                    found_position = True
                    # print(f'position is {position}')
                else:
                    name = str(b)
                    # print(f'name is {name}')

            if found_position == False:
                position = 'member'
            else:
                position = position_ext
                
            member = pd.DataFrame({'name':[name], 'district': [district_number], 'position':[position]})
            print(member.to_string())
            members_bucket.append(member)
            

    com_df = pd.concat(members_bucket).reset_index(drop = True)
    print(com_df.to_string())

    com_name = url.split('/')[-1].replace(r'-', ' ').title()
    com_df['com_name'] = com_name
    if 'senate' in str(url):
        chamber = 'Senate'
    else:
        chamber = 'House'
    
    com_df['chamber'] = chamber


    print(com_df.to_string())
    oh_coms.append(com_df)


house_ext_df = pd.concat(oh_coms).reset_index(drop = True)
house_ext_df = house_ext_df[['com_name', 'chamber', 'name', 'district', 'position']]



In [128]:
#senate

oh_sen_com_urls = '''https://ohiosenate.gov/committees/education
https://ohiosenate.gov/committees/health
https://ohiosenate.gov/committees/higher-education
https://ohiosenate.gov/committees/ways-and-means
'''
oh_sen_com_urls = [x for x in oh_sen_com_urls.split('\n') if len(x) != 0]

oh_sen_coms = []
for url in oh_sen_com_urls:
    print(f'going to {url}')
    driver.get(url)

    html_from_page = driver.page_source
    soup = BeautifulSoup(html_from_page, 'html.parser')
    hrefs = soup.find_all("a", href = True)
    members_bucket = []
    for x in hrefs:
        
        if 'members' in str(x.get("href")).lower()  and 'target="_self"' in str(x).lower():
            

            # text = x.text.encode('unicode_escape').decode('utf-8')
            text_split = x.text.split('\n')

            # print(text_split)

            found_position = False
            for i,b in enumerate(text_split):
                position = np.nan
                words = ['chair', 'vice', 'ranking', 'member']
                
                #covers len of 0
                if len(b) == 0:
                    continue
                            #covers len of 0
                elif '\xa0' in str(b).strip() and len(str(b).strip())<5:
                    continue

                #covers district sections
                elif 'district' in str(b).lower().strip():
                    match = re.search(r"[Dd]istrict\s+(\d+)", str(b))

                    if match:
                        district_number = match.group(1)  # Extract the number
                        # print(district_number)  # Output: 36
                    else:
                        print("No match found")
                #covers positions
                elif any(word in str(b).lower().strip() for word in words):
                    position_ext = str(b)
                    found_position = True
                    # print(f'position is {position}')
                else:
                    name = str(b)
                    # print(f'name is {name}')

            if found_position == False:
                position = 'member'
            else:
                position = position_ext
                
            member = pd.DataFrame({'name':[name], 'district': [district_number], 'position':[position]})
            members_bucket.append(member)
            

    com_df = pd.concat(members_bucket).reset_index(drop = True)
    # print(com_df.to_string())

    com_name = url.split('/')[-1].replace(r'-', ' ').title()
    com_df['com_name'] = com_name
    com_df['chamber'] = 'Senate'


    print(com_df.to_string())
    oh_sen_coms.append(com_df)


senate_ext_df = pd.concat(oh_sen_coms).reset_index(drop = True)
senate_ext_df = senate_ext_df[['com_name', 'chamber', 'name', 'district', 'position']]



going to https://ohiosenate.gov/committees/education
                     name district        position   com_name chamber
0       Andrew O. Brenner       19           Chair  Education  Senate
1  Louis W. Blessing, III        8      Vice Chair  Education  Senate
2     Catherine D. Ingram        9  Ranking Member  Education  Senate
3              Al Cutrona       33          member  Education  Senate
4      Stephen A. Huffman        5          member  Education  Senate
5            Kyle Koehler       10          member  Education  Senate
6              Kent Smith       21          member  Education  Senate
going to https://ohiosenate.gov/committees/health
                  name district        position com_name chamber
0   Stephen A. Huffman        5           Chair   Health  Senate
1        Terry Johnson       14      Vice Chair   Health  Senate
2          Beth Liston       16  Ranking Member   Health  Senate
3  Catherine D. Ingram        9          member   Health  Senate
4     Susan 

In [130]:




ext_df = pd.concat([house_ext_df, senate_ext_df]).reset_index(drop=True)
ext_df.to_excel(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\OH_coms.xlsx', index=False)



# senate_ext_df

In [ ]:

for k,v in oh_com_dict.items():
    print(f'{k}:{v}')

Children And Human Services:                   name district position com_name
0          Andrea White       36   member    House
1            Jodi Salvo       51   member    House
2          Crystal Lett       11   member    House
3     Darnell T. Brewer       22   member    House
4        Karen Brownlee       28   member    House
5            Gary Click       88   member    House
6         Kellie Deeter       54   member    House
7   Sarah Fowler Arthur       99   member    House
8        Melanie Miller       67   member    House
9           Mike Odioso       30   member    House
10  Beryl Piccolantonio        4   member    House
11        Sharon A. Ray       66   member    House
12  Tracy M. Richardson       86   member    House
Education:                        name district position com_name
0        Sarah Fowler Arthur       99   member    House
1                Mike Odioso       30   member    House
2   Phillip M. Robinson, Jr.       19   member    House
3               Adam C. 

In [40]:
import camelot
os.getcwd()
file = get_recent_file('*',r"C:\Users\clutz\Downloads")
print(file)
# data = pd.read_clipboard()
# tables = camelot.read_pdf(file)


C:\Users\clutz\Downloads\cmte_h_list.pdf


In [41]:

import PyPDF2

def pdf_to_text(pdf_path, output_txt):
    # Open the PDF file in read-binary mode
    with open(pdf_path, 'rb') as pdf_file:
        # Create a PdfReader object instead of PdfFileReader
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Initialize an empty string to store the text
        text = ''

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Write the extracted text to a text file
    with open(output_txt, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)



In [ ]:
def reset_eof_of_pdf_return_stream(pdf_stream_in:list):
    # find the line position of the EOF
    for i, x in enumerate(txt[::-1]):
        if b'%%EOF' in x:
            actual_line = len(pdf_stream_in)-i
            print(f'EOF found at line position {-i} = actual {actual_line}, with value {x}')
            break

    # return the list up to that point
    return pdf_stream_in[:actual_line

In [42]:
pdf_to_text(file, "ohio_house_coms.txt")


In [43]:



with open('ohio_house_coms.txt', 'r') as file:
    for line in file:
        # Print each line
        
        print(f'length({len(line.strip())}){line.strip()}')  # Remove leading/trailing whitespace



length(0)
length(0)
length(0)
length(0)
length(20)STANDING  AND SELECT
length(10)COMMITTEES
length(0)
length(6)OF THE
length(31)OHIO  HOUSE  OF REPRESENTATIVES
length(0)
length(0)
length(23)136TH GENERAL  ASSEMBLY
length(0)
length(0)
length(0)
length(0)
length(5)AS OF
length(17)FEBRUARY 21, 2025
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(0)
length(24)MESSAGE FROM THE SPEAKER
length(79)Pursuant to House Rules 13, 30, and 3 2, the Speaker hereby makes the following
length(42)appointments to House standing committees:
length(0)
length(0)
length(0)
length(0)
length(11)AGRICULTURE
length(42)Creech , Chair  Miller, J.  Ranking Member
length(26)Newman , Vice Chair  Brent
length(17)Barhorst  Cockley
length(10)Dean  Sims
length(5)Hiner
length(5)Jones
length(12)Klopfenstein
length(7)Mullins
length(8)Peterson
length(0)
length(28)ARTS, ATHLETICS, AND TOURI

In [26]:
# print(tables[0].data)
for table in tables[0].data:
    print(table)

['2025 SESSION—SENATE COMMITTEES\nTIME AND PLACE OF MEETINGS', '', '', '']
['', 'RULES\nSenate Room C (Room 311), General Assembly Building\nUpon Call of the Chair\nSENATORS\nMARSDEN\nMCPIKE\nROUSE\nSTUART\nLOCKE, Chair\nLUCAS\nFAVOLA\nSUROVELL\nBAGBY\nPEAKE\nDEEDS\nEBBIN\nHASHMI\nMCDOUGLE\nPILLION\nSUSAN CLARKE SCHAAR, Clerk of the Senate, and HOBIE LEHMAN', '', '']
['SENATE STANDING COMMITTEES\nALL COMMITTEES WILL MEET IN THE GENERAL ASSEMBLY BUILDING UNLESS OTHERWISE NOTIFIED', '', '', '']
['MONDAY\nWEDNESDAY', '', '', '']
['SENATORS\nSUROVELL, Chair\nOBENSHAIN\nMCDOUGLE\nDEEDS\nSTUART\nSENATORS\nMCPIKE, Chair\nSTANLEY\nHACKWORTH\nSUETTERLEIN\nSTURTEVANT\nCOMMERCE AND LABOR\nSenate Room A (Room 305), General Assembly Building\n15 minutes after adjournment', 'COURTS OF JUSTICE\nSenate Room A (Room 305), General Assembly Building\n8:00 a.m.\nSTANLEY\nAIRD\nBOYSKO\nCARROLL FOY\nPEAKE\nPERRY\nBAGBY\nSALIM\nJORDAN\nFAVOLA\nClerk: MICHAEL JACKSON\nLOCAL GOVERNMENT\nSenate Room B (Room 306